In [1]:
import lyra_graphtool_test as lgtool
from LS_utils import ORIGN, BASIC_SITE, SITE1, SITE2,  SITE3, Node, Graph, Worker
import random
from copy import deepcopy
import itertools

In [2]:
json_filename = "./graphs/test_graphs/graph1.json"
graph = Graph(json_filename=json_filename)

In [3]:
class LocalSearchAgent() :
    def __init__(self, graph : Graph) -> None:
        self.steps = 20
        self.processed = False
        self.graph: Graph = graph
        self.origin = graph.get_Origin().get_coordinate()
        self.BUDGET = 10000.00
        self.PROFIT = 0
        self.worker_list = list()
        self.worker_schedule = list()
        self.required_vertices_1 = 0
        self.required_vertices_2 = 0
        self.required_vertices_3 = 0
        self.shortest_path = self.floyd_warshall(graph)
        self.specific_shortest_path = self.preprocess(self.shortest_path)
        self.epsilon = 0.4

        # print(self.shortest_path)
        # print(self.specific_shortest_path)

    def floyd_warshall(self, graph):
        inf = float('inf')
        vertices = list(self.graph.get_vertices().keys())
        num_vertices = len(vertices)
        distance = [[inf for _ in range(num_vertices)] for _ in range(num_vertices)]

        for i in range(num_vertices):
            distance[i][i] = 0

        vertices_index = dict()
        for i in range(num_vertices):
            vertices_index[vertices[i]] = i
        
        edges = self.graph.get_edges()    
        for i in range(num_vertices):
            for edge in edges[vertices[i]]:
                distance[i][vertices_index[edge]] = 1

        for k in range(num_vertices):
            for i in range(num_vertices):
                for j in range(num_vertices):
                    curr_distance = distance[i][k] + distance[k][j]
                    if distance[i][j] > curr_distance:
                        distance[i][j] = curr_distance

        return distance

    def preprocess(self, shortest_path):
        vertices = list(self.graph.get_vertices().keys())
        num_vertices = len(vertices)

        vertices_index = dict()
        shortest_distance = dict()
        vertices_required = list()
        
        for i in range(num_vertices):
            vertices_index[vertices[i]] = i
            
            if self.graph.is_reward_site_by_node(self.graph.get_vertices()[vertices[i]]):
                shortest_distance[vertices[i]] = {1: list(), 2: list(), 3: list()}
                vertices_required.append(i)

        for i in vertices_required:
            temp = shortest_path[i]
            for j in range(num_vertices):
                if j not in vertices_required:
                    continue
                if i == j:
                    continue
                vertex_type = self.graph.get_Node(*vertices[j]).get_type() - 1
                if vertex_type != 0:
                    shortest_distance[vertices[i]][self.graph.get_Node(*vertices[j]).get_type() - 1].append([vertices[j], shortest_path[i][j]])
            for k in [1,2,3]:
                shortest_distance[vertices[i]][k].sort(key = lambda x: x[1])

        for i in vertices_required:
            x = self.graph.get_Node(*vertices[i]).get_type() - 1
            if x == 1:
                self.required_vertices_1 += 1
            elif x == 2:
                self.required_vertices_2 += 1
            elif x == 3:
                self.required_vertices_3 += 1

        return shortest_distance

    def simulate(self, worker_list):
        origin = self.graph.get_Origin()
        curr_schedule = list()

        # 2D
        all_schedules = list()
        
        reward = [0]
        cost = [0]
        
        for i in range(len(worker_list)):
            worker_num = worker_list[i]
            curr_schedule.append(Worker(worker_num, self.graph.get_Origin(), self.graph.get_worker_cost(worker_num), 0))
            cost[0] += self.graph.get_worker_cost(worker_num)

        curr_schedule = [curr_schedule]
        all_schedules.append(curr_schedule)
        if cost[0] > self.BUDGET:
            return None, 0, 0, cost[0]

        visited = list()
        for ts in range(1, self.steps):
            temp_schedule = list()
            temp_cost = cost[-1]
            temp_reward = reward[-1]
            for w in all_schedules[-1][-1]:
                if w.get_timestamp() > ts:
                    temp_schedule.append(Worker(w.get_type(), w.get_location(), w.get_rate(), w.get_timestamp()))
                    continue

                distances_1 = self.specific_shortest_path[w.get_location().get_coordinate()][1]
                distances_2 = self.specific_shortest_path[w.get_location().get_coordinate()][2]
                distances_3 = self.specific_shortest_path[w.get_location().get_coordinate()][3]

                # Clear all sites already booked
                distances_1 = list(filter(lambda x: x[0] not in visited, distances_1))
                distances_2 = list(filter(lambda x: x[0] not in visited, distances_2))
                distances_3 = list(filter(lambda x: x[0] not in visited, distances_3))

                # print("Visited:", visited, "D1:", distances_1, "D2:", distances_2, "D3:", distances_3)

                # Continue when no more sites left
                if len(distances_1) == 0 and len(distances_2) == 0 and len(distances_3) == 0:
                    continue

                EMPTY_BEST = [(0,0), self.steps + 1]

                # Get closest site
                best_1 = distances_1[0] if len(distances_1) > 0 else EMPTY_BEST
                best_2 = distances_2[0] if len(distances_2) > 0 else EMPTY_BEST
                best_3 = distances_3[0] if len(distances_3) > 0 else EMPTY_BEST

                EMPTY_NODE = Node({"vertex_type":0,"reward":0,"mult_time_active":0,"time_to_acquire":self.steps+1}, (0,0))
                
                # Get node of closest site
                next_node_1 = self.graph.get_Node(*best_1[0]) if best_1 is not EMPTY_BEST else EMPTY_NODE
                next_node_2 = self.graph.get_Node(*best_2[0]) if best_2 is not EMPTY_BEST else EMPTY_NODE
                next_node_3 = self.graph.get_Node(*best_3[0]) if best_3 is not EMPTY_BEST else EMPTY_NODE

                # If not enough time to reach site and extract, no closest site
                if best_1[1] + next_node_1.get_acquire_time() + ts > self.steps:
                    best_1 = EMPTY_BEST
                if best_2[1] + next_node_2.get_acquire_time() + ts > self.steps:
                    best_2 = EMPTY_BEST
                if best_3[1] + next_node_3.get_acquire_time() + ts > self.steps:
                    best_3 = EMPTY_BEST

                # If exceed budget, no closest site
                if (next_node_1 is EMPTY_NODE) and (temp_cost + w.get_rate() * best_1[1] > self.BUDGET):
                    next_node_1 = EMPTY_NODE
                if (next_node_2 is EMPTY_NODE) and (temp_cost + w.get_rate() * best_2[1] > self.BUDGET):
                    next_node_2 = EMPTY_NODE
                if (next_node_3 is EMPTY_NODE) and (temp_cost + w.get_rate() * best_3[1] > self.BUDGET):
                    next_node_3 = EMPTY_NODE

                # Continue when no more sites left
                if next_node_1 is EMPTY_NODE and next_node_2 is EMPTY_NODE and next_node_3 is EMPTY_NODE:
                    continue

                NEG_INF = float('-inf')

                # Calculate profit of each best site
                profit_1 = next_node_1.get_reward() - w.get_rate() * best_1[1] if next_node_1 is not EMPTY_NODE else NEG_INF
                profit_2 = next_node_2.get_reward() - w.get_rate() * best_2[1] if next_node_2 is not EMPTY_NODE else NEG_INF
                profit_3 = next_node_3.get_reward() - w.get_rate() * best_3[1] if next_node_3 is not EMPTY_NODE else NEG_INF
                profit_arr = [profit_1, profit_2, profit_3]

                def helper(w, s, v):
                    match w.get_type():
                        case 1:
                            s.append(Worker(w.get_type(), next_node_1, w.get_rate(), best_1[1] + next_node_1.get_acquire_time() + ts))
                            v.append(best_1[0])
                            return w.get_rate() * best_1[1], next_node_1.get_reward()
                        case 2:
                            s.append(Worker(w.get_type(), next_node_2, w.get_rate(), best_2[1] + next_node_2.get_acquire_time() + ts))
                            v.append(best_2[0])
                            return w.get_rate() * best_2[1], next_node_2.get_reward()
                        case 3:
                            s.append(Worker(w.get_type(), next_node_3, w.get_rate(), best_3[1] + next_node_3.get_acquire_time() + ts))
                            v.append(best_3[0])
                            return w.get_rate() * best_3[1], next_node_3.get_reward()
                        case _:
                            return 0, 0

                profit_arr_n = profit_arr[0:w.get_type()]
                max_profit = max(profit_arr_n)
                if max_profit >= 0:
                    max_profit_index = profit_arr_n.index(max_profit)
                    c, r = helper(w, temp_schedule, visited)
                    temp_cost += c
                    temp_reward += r
            
            new_schedule = deepcopy(all_schedules[-1])
            new_schedule.append(temp_schedule)
            all_schedules.append(new_schedule)
            reward.append(temp_reward)
            cost.append(temp_cost)
        
        profit = list(map(lambda x: x[0] - x[1], zip(reward, cost)))
        best = profit.index(max(profit))
        return all_schedules[best], profit[best], reward[best], cost[best]

    def add_actions(self, worker_list):
        new_worker_list = list()
        
        if worker_list.count(1) < self.required_vertices_1:
            # Hire_1
            temp_1 = deepcopy(worker_list)
            temp_1.append(1)
            random.shuffle(temp_1)
            new_worker_list.append(temp_1)

        if worker_list.count(2) < self.required_vertices_2 or worker_list.count(1) + worker_list.count(2) < self.required_vertices_1 + self.required_vertices_2:
            # Hire_2
            temp_2 = deepcopy(worker_list)
            temp_2.append(2)
            random.shuffle(temp_2)
            new_worker_list.append(temp_2)

        if worker_list.count(3) < self.required_vertices_3 or worker_list.count(1) + worker_list.count(2) + worker_list.count(3) < self.required_vertices_1 + self.required_vertices_2 + self.required_vertices_3:
            # Hire_3
            temp_3 = deepcopy(worker_list)
            temp_3.append(3)
            random.shuffle(temp_3)
            new_worker_list.append(temp_3)
        
        if 3 in worker_list:
            if worker_list.count(1) < self.required_vertices_1:
                # Replace_1
                temp_4 = deepcopy(worker_list)
                temp_4.remove(3)
                temp_4.append(1)
                random.shuffle(temp_3)
                new_worker_list.append(temp_4)
            if worker_list.count(2) < self.required_vertices_2 or worker_list.count(1) + worker_list.count(2) < self.required_vertices_1 + self.required_vertices_2:
                # Replace_2
                temp_5 = deepcopy(worker_list)
                temp_5.remove(3)
                temp_5.append(2)
                random.shuffle(temp_5)
                new_worker_list.append(temp_5)

        if 2 in worker_list:
            if worker_list.count(1) < self.required_vertices_1:
                # Replace_1
                temp_6 = deepcopy(worker_list)
                temp_6.remove(2)
                temp_6.append(1)
                random.shuffle(temp_6)
                new_worker_list.append(temp_6)
            if worker_list.count(3) < self.required_vertices_3 or worker_list.count(1) + worker_list.count(2) + worker_list.count(3) < self.required_vertices_1 + self.required_vertices_2 + self.required_vertices_3:
                # Replace_3
                temp_7 = deepcopy(worker_list)
                temp_7.remove(2)
                temp_7.append(3)
                random.shuffle(temp_7)
                new_worker_list.append(temp_7)

        if 1 in worker_list:
            if worker_list.count(2) < self.required_vertices_2 or worker_list.count(1) + worker_list.count(2) < self.required_vertices_1 + self.required_vertices_2:
                # Replace_2
                temp_8 = deepcopy(worker_list)
                temp_8.remove(1)
                temp_8.append(2)
                random.shuffle(temp_8)
                new_worker_list.append(temp_8)
            if worker_list.count(3) < self.required_vertices_3 or worker_list.count(1) + worker_list.count(2) + worker_list.count(3) < self.required_vertices_1 + self.required_vertices_2 + self.required_vertices_3:
                # Replace_3
                temp_9 = deepcopy(worker_list)
                temp_9.remove(1)
                temp_9.append(3)
                random.shuffle(temp_9)
                new_worker_list.append(temp_9)

        return new_worker_list

    # Hill climbing
    def process_hc(self):
        # print(self.required_vertices_1, self.required_vertices_2, self.required_vertices_3)
        worker_list = [3]

        history_worker_list = [worker_list]
        s = self.simulate(worker_list)
        history_profit = [s[1]]

        history = dict()
        history_wl = set()
        history_wl.add(tuple(worker_list))
        history[tuple(worker_list)] = s

        # MAX_SIZE = (2**self.required_vertices_1)*(2**self.required_vertices_2)*(2**self.required_vertices_3)
        # counter = 1
        
        while True:
            # print(counter)
            # counter += 1
            next_actions = self.add_actions(worker_list)
            if len(next_actions) == 0:
                break

            all_seen = True
            for n in next_actions:
                if tuple(n) not in history_wl:
                    all_seen = False
                    break
            if all_seen:
                break
            
            profit = list()
            for l in next_actions:
                if tuple(l) in history_wl:
                    profit.append(history[tuple(l)][1])
                else:
                    s = self.simulate(l)
                    history_wl.add(tuple(l))
                    history[tuple(l)] = s
                    profit.append(s[1])
            biggest = profit.index(max(profit))
            if profit[biggest] <= 0:
                break
            else:
                history_worker_list.append(next_actions[biggest])
                history_profit.append(profit[biggest])
                worker_list = next_actions[biggest]
                # print(profit[biggest], next_actions[biggest])

        self.processed = True
        best = history_profit.index(max(history_profit))
        self.PROFIT = history_profit[best]
        self.worker_list = history_worker_list[best]
        self.worker_schedule = history[tuple(history_worker_list[best])][0]

        return self.PROFIT, self.worker_list, self.worker_schedule

    # Hill climbing (Random Restart)
    def process_hc_rr(self):
        # print(self.required_vertices_1, self.required_vertices_2, self.required_vertices_3)
        outside_wl = list()
        outside_profit = list()
        outside_schedule = list()

        for i in range(10):
            worker_list = list()
            for _ in range(self.required_vertices_1):
                if random.random() < self.epsilon:
                    worker_list.append(1)
            for _ in range(self.required_vertices_2):
                if random.random() < self.epsilon:
                    worker_list.append(2)
            for _ in range(self.required_vertices_3):
                if random.random() < self.epsilon:
                    worker_list.append(3)

            random.shuffle(worker_list)
            # print(worker_list)
        
            history_worker_list = [worker_list]
            s = self.simulate(worker_list)
            history_profit = [s[1]]
    
            history = dict()
            history_wl = set()
            history_wl.add(tuple(worker_list))
            history[tuple(worker_list)] = s
            
            while True:
                next_actions = self.add_actions(worker_list)
                for i in next_actions:
                    random.shuffle(i)
                
                if len(next_actions) == 0:
                    break

                all_seen = True
                for n in next_actions:
                    if tuple(n) not in history_wl:
                        all_seen = False
                        break
                if all_seen:
                    break
                
                profit = list()
                for l in next_actions:
                    if tuple(l) in history_wl:
                        profit.append(history[tuple(l)][1])
                    else:
                        s = self.simulate(l)
                        history_wl.add(tuple(l))
                        history[tuple(l)] = s
                        profit.append(s[1])
                biggest = profit.index(max(profit))
                if profit[biggest] <= 0:
                    break
                else:
                    history_worker_list.append(next_actions[biggest])
                    history_profit.append(profit[biggest])
                    worker_list = next_actions[biggest]
                    # print(profit[biggest], next_actions[biggest])

            best = history_profit.index(max(history_profit))
            outside_profit.append(history_profit[best])
            outside_wl.append(history_worker_list[best])
            outside_schedule.append(history[tuple(history_worker_list[best])][0])

        self.processed = True
        best = outside_profit.index(max(outside_profit))
        self.PROFIT = outside_profit[best]
        self.worker_list = outside_wl[best]
        self.worker_schedule = outside_schedule[best]

        return self.PROFIT, self.worker_list, self.worker_schedule

    # Hill climbing (Beam)
    def process_hc_b(self):
        # print(self.required_vertices_1, self.required_vertices_2, self.required_vertices_3)
        worker_list = [3]

        history_worker_list = [worker_list]
        s = self.simulate(worker_list)
        history_profit = [s[1]]

        history = dict()
        history_wl = set()
        history_wl.add(tuple(worker_list))
        history[tuple(worker_list)] = s

        worker_list_2 = [3]
        
        while True:
            next_actions = self.add_actions(worker_list)
            next_actions.extend(self.add_actions(worker_list_2))
            if len(next_actions) == 0:
                break

            all_seen = True
            for n in next_actions:
                if tuple(n) not in history_wl:
                    all_seen = False
                    break
            if all_seen:
                break
            
            profit = list()
            for l in next_actions:
                if tuple(l) in history_wl:
                    profit.append(history[tuple(l)][1])
                else:
                    s = self.simulate(l)
                    history_wl.add(tuple(l))
                    history[tuple(l)] = s
                    profit.append(s[1])
            biggest = profit.index(max(profit))
            if profit[biggest] <= 0:
                break
            else:
                history_worker_list.append(next_actions[biggest])
                history_profit.append(profit[biggest])
                worker_list = next_actions[biggest]
                # print(profit[biggest], next_actions[biggest])

            del profit[biggest]
            del next_actions[biggest]
            
            biggest = profit.index(max(profit))
            if profit[biggest] <= 0:
                break
            else:
                history_worker_list.append(next_actions[biggest])
                history_profit.append(profit[biggest])
                worker_list_2 = next_actions[biggest]
                # print(profit[biggest], next_actions[biggest])

        self.processed = True
        best = history_profit.index(max(history_profit))
        self.PROFIT = history_profit[best]
        self.worker_list = history_worker_list[best]
        self.worker_schedule = history[tuple(history_worker_list[best])][0]

        return self.PROFIT, self.worker_list, self.worker_schedule

    # Brute force through all states
    def process_bf(self):
        temp_worker_list = [[]]
        for i in range(self.required_vertices_1):
            temp_list = list()
            for w in temp_worker_list:
                temp = deepcopy(w)
                temp.append(1)
                temp_list.append(temp)
            temp_worker_list.extend(temp_list)
        for i in range(self.required_vertices_2):
            temp_list = list()
            for w in temp_worker_list:
                temp = deepcopy(w)
                temp.append(2)
                temp_list.append(temp)
            temp_worker_list.extend(temp_list)
        for i in range(self.required_vertices_3):
            temp_list = list()
            for w in temp_worker_list:
                temp = deepcopy(w)
                temp.append(3)
                temp_list.append(temp)
            temp_worker_list.extend(temp_list)
            
        worker_list = list()
        for x in temp_worker_list:
            worker_list.extend( list(itertools.permutations(x)))
       
        simulate_result = list()
        profit = list()
        
        for wl in worker_list:
            s = self.simulate(wl)
            simulate_result.append(s)
            profit.append(s[1])

        self.processed = True
        best = profit.index(max(profit))
        self.PROFIT = profit[best]
        self.worker_list = worker_list[best]
        self.worker_schedule = simulate_result[best][0]

        return self.PROFIT, self.worker_list, self.worker_schedule

    def get_profit(self):
        if not self.processed:
            self.process()
        return self.PROFIT

    def get_worker_list(self):
        if not self.processed:
            self.process()
        return self.worker_list


In [4]:
lsa = LocalSearchAgent(graph)

In [5]:
lsa.process_hc()

(6600.0,
 [1, 3, 2, 3],
 [[Worker type 1 at ((1, 9) TYPE : 1) at ts 0,
   Worker type 3 at ((1, 9) TYPE : 1) at ts 0,
   Worker type 2 at ((1, 9) TYPE : 1) at ts 0,
   Worker type 3 at ((1, 9) TYPE : 1) at ts 0],
  [Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 1 at ((4

In [6]:
lsa.process_hc_rr()

(6600.0,
 [1, 3, 2, 3],
 [[Worker type 1 at ((1, 9) TYPE : 1) at ts 0,
   Worker type 3 at ((1, 9) TYPE : 1) at ts 0,
   Worker type 2 at ((1, 9) TYPE : 1) at ts 0,
   Worker type 3 at ((1, 9) TYPE : 1) at ts 0],
  [Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 1 at ((4

In [7]:
lsa.process_hc_b()

(6600.0,
 [2, 3, 1, 3],
 [[Worker type 2 at ((1, 9) TYPE : 1) at ts 0,
   Worker type 3 at ((1, 9) TYPE : 1) at ts 0,
   Worker type 1 at ((1, 9) TYPE : 1) at ts 0,
   Worker type 3 at ((1, 9) TYPE : 1) at ts 0],
  [Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 2 at ((2, 6) TYPE : 3) at ts 7,
   Worker type 3 at ((4, 8) TYPE : 4) at ts 10,
   Worker type 1 at ((4, 4) TYPE : 2) at ts 8,
   Worker type 3 at ((0, 4) TYPE : 4) at ts 12],
  [Worker type 2 at ((2

In [ ]:
bf = list()
hc = list()
hc_rr = list()
hc_b = list()
for i in range(0, 100):
    print(i+1)
    json_filename = f"./graphs/test_graphs/graph{i+1}.json"
    graph = Graph(json_filename=json_filename)
    lsa = LocalSearchAgent(graph)
    bf.append(lsa.process_bf())
    hc.append(lsa.process_hc())
    hc_rr.append(lsa.process_hc_rr())
    hc_b.append(lsa.process_hc_b())

print("Profit")
print("Brute Force:", [x[0] for x in bf], "HC:", [x[0] for x in hc], "HC (RR):", [x[0] for x in hc_rr], "HC (B):", [x[0] for x in hc_b])
print("Worker List")
print("Brute Force:", [x[1] for x in bf], "HC:", [x[1] for x in hc], "HC (RR):", [x[1] for x in hc_rr], "HC (B):", [x[1] for x in hc_b])

1
